In [ ]:
%pylab inline
import calin.ix.scripts.compute_diagnostics
import calin.io.sql_transceiver
import calin.diagnostics.waveform
import calin.diagnostics.functional
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
sql = calin.io.sql_transceiver.SQLite3Transceiver("/CTA/diagnostics.sqlite",
  calin.io.sql_transceiver.SQLite3Transceiver.READ_ONLY)
diag = calin.ix.scripts.compute_diagnostics.Results()
sql.retrieve_by_oid("diagnostics_results", 1, diag)
del sql

In [ ]:
wfs  = diag.waveform_stats()
psd  = diag.waveform_psd()
sig  = diag.sig_stats()
bkg  = diag.bkg_stats()
smb  = diag.sig_minus_bkg_stats()
t0   = diag.t0_stats()
conf = diag.run_config()

In [ ]:
def plot_histogram(h,*args):
    hx = h.xval0()+h.dxval()*frange(0,h.bins_size(),closed=0)
    hy = h.bins_view()
    return step(hx,hy, where='post', *args)

In [ ]:
with PdfPages('/CTA/diagnostics.pdf') as pdf:
    figure(figsize=(8.27, 11.69),dpi=300)
    gs = GridSpec(4, 2,
            height_ratios=[1,1,1.5,1.5],
            width_ratios=[1,1],
            hspace=0.4,
            wspace=0.3,
            bottom=0.05,
            top=0.9
          )

    figtext(0.5,0.97,'--- All channels ---',va='top',ha='center',fontsize=16)
    figtext(0.3,0.94,'-- High gain -- ',va='top',ha='center',fontsize=14)
    figtext(0.725,0.94,'-- Low gain --',va='top',ha='center',fontsize=14)
    
    lg_wf_mean = zeros(conf.num_samples())
    hg_wf_mean = zeros(conf.num_samples())
    for ich in  range(0,wfs.high_gain_size()):
        hg_wf_mean += calin.diagnostics.waveform.WaveformStatsVisitor.waveform_mean(wfs.high_gain(ich))
        lg_wf_mean += calin.diagnostics.waveform.WaveformStatsVisitor.waveform_mean(wfs.low_gain(ich))

    subplot(gs[0,0])
    title("All channel mean waveform")
    ylabel('Mean waveform [DC]')
    xlabel('Sample number')
    plot(hg_wf_mean/wfs.high_gain_size())
    
    subplot(gs[0,1])
    title("All channel mean waveform")
    ylabel('Mean waveform [DC]')
    xlabel('Sample number')
    plot(lg_wf_mean/wfs.high_gain_size())    
    
    subplot(gs[1,0])
    title("Mean signal minus background")
    ylabel('Signal [DC samples]')
    xlabel('Channel index')
    xmean = calin.diagnostics.functional.channel_mean_int(smb.high_gain())
    xrms = sqrt(calin.diagnostics.functional.channel_var_int(smb.high_gain()))
    errorbar(frange(0,len(xmean),closed=False),xmean,xrms,fmt='.')

    subplot(gs[1,1])
    title("Mean signal minus background")
    ylabel('Signal [DC samples]')
    xlabel('Channel index')
    xmean = calin.diagnostics.functional.channel_mean_int(smb.low_gain())
    xrms = sqrt(calin.diagnostics.functional.channel_var_int(smb.low_gain()))
    errorbar(frange(0,len(xmean),closed=False),xmean,xrms,fmt='.')

    subplot(gs[2,0])
    h=pcolor(calin.diagnostics.functional.channel_cov_frac_int(smb.high_gain()),
            cmap=get_cmap('jet'));
    title("Signal minus background cov matrix")
    xlabel('Channel index')
    ylabel('Channel index')
#    h.set_clim((-1.0,1.0))
    cb = colorbar()
#    cb.set_label("Covariance fraction")
    axis('square')

    subplot(gs[2,1])
    h=pcolor(calin.diagnostics.functional.channel_cov_frac_int(smb.low_gain()),
            cmap=get_cmap('jet'));
    title("Signal minus background cov matrix")
    xlabel('Channel index')
    ylabel('Channel index')
#    h.set_clim((-1.0,1.0))
    cb = colorbar()
#    cb.set_label("Covariance fraction")
    axis('square')

    subplot(gs[3,0])
    h=pcolor(calin.diagnostics.functional.channel_cov_frac_int(bkg.high_gain()),
            cmap=get_cmap('jet'));
    title("Background cov matrix")
    xlabel('Channel index')
    ylabel('Channel index')
#    h.set_clim((-1.0,1.0))
    cb = colorbar()
#    cb.set_label("Covariance fraction")
    axis('square')

    subplot(gs[3,1])
    h=pcolor(calin.diagnostics.functional.channel_cov_frac_int(bkg.low_gain()),
            cmap=get_cmap('jet'));
    title("Background cov matrix")
    xlabel('Channel index')
    ylabel('Channel index')
#    h.set_clim((-1.0,1.0))
    cb = colorbar()
#    cb.set_label("Covariance fraction")
    axis('square')

    pdf.savefig()
    close(gcf())
    
    for ich in range(0,wfs.high_gain_size()):
        hg_wfs = wfs.high_gain(ich)
        hg_psd = psd.high_gain(ich)
        hg_smb = smb.high_gain().value_hist(ich)
        hg_t0  = t0.high_gain().value_hist(ich)
        lg_wfs = wfs.low_gain(ich)
        lg_psd = psd.low_gain(ich)
        lg_smb = smb.low_gain().value_hist(ich)
        lg_t0  = t0.low_gain().value_hist(ich)
        
        chid = conf.configured_channel_id(ich)
        ch = conf.camera_layout().channel(chid)

        figure(figsize=(8.27, 11.69),dpi=300)
        gs = GridSpec(5, 2,
                height_ratios=[1,1,1,1.5,1],
                width_ratios=[1,1],
                hspace=0.4,
                wspace=0.3,
                bottom=0.05,
                top=0.9
              )

        figtext(0.5,0.97,'--- Channel %d (mod: %d/%d) ---'%(ch.channel_index(),
            ch.module_index(),ch.module_channel_index()),va='top',ha='center',fontsize=16)
        figtext(0.3,0.94,'-- High gain -- ',va='top',ha='center',fontsize=14)
        figtext(0.725,0.94,'-- Low gain --',va='top',ha='center',fontsize=14)

        subplot(gs[0,0])
        title("Mean waveform")
        ylabel('Mean waveform [DC]')
        xlabel('Sample number')
        plot(calin.diagnostics.waveform.WaveformStatsVisitor.waveform_mean(hg_wfs))
        a=axis()
        if(a[3] >= 4095):
            axhline(y=4095,linestyle='--',color='k')

        subplot(gs[0,1])
        title("Mean waveform")
        ylabel('Mean waveform [DC]')
        xlabel('Sample number')
        plot(calin.diagnostics.waveform.WaveformStatsVisitor.waveform_mean(lg_wfs))
        a=axis()
        if(a[3] >= 4095):
            axhline(y=4095,linestyle='--',color='k')

        subplot(gs[1,0])
        title("Signal minus background")
        ylabel('Frequency [events/DC]')
        xlabel('Integrated charge [DC samples]')
        plot_histogram(hg_smb)

        subplot(gs[1,1])
        title("Signal minus background")
        ylabel('Frequency [events/DC]')
        xlabel('Integrated charge [DC samples]')
        plot_histogram(lg_smb)

        subplot(gs[2,0])
        title("Pulse start time (T0)")
        ylabel('Frequency [events/DC]')
        xlabel('Pulse start time [samples]')
        plot_histogram(hg_t0)

        subplot(gs[2,1])
        title("Pulse start time (T0)")
        ylabel('Frequency [events/DC]')
        xlabel('Pulse start time [samples]')
        plot_histogram(lg_t0)

        subplot(gs[4,0])
        f = asarray(range(0,hg_psd.psd_sum_size()))/60*1000
        semilogy(f, calin.diagnostics.waveform.WaveformPSDVisitor.psd_mean(hg_psd))
        title("High-gain PSD")
        ylabel('Signal squared [DC^2]')
        xlabel('Frequency [MHz]')

        subplot(gs[4,1])
        c, x = calin.diagnostics.waveform.WaveformPSDVisitor.corr_mean_centered(hg_psd, hg_wfs)
        plot(x,c)
        title("High-gain auto correlation")
#        ylabel('Signal squared [DC^2]')
        xlabel('Time shift [samples]')

        subplot(gs[3,0])
        h=pcolor(calin.diagnostics.waveform.WaveformStatsVisitor.waveform_cov_frac(hg_wfs),
                cmap=get_cmap('bwr'));
        title("Waveform covariance matrix")
        xlabel('Sample number')
        ylabel('Sample number')
        h.set_clim((-1.0,1.0))
        cb = colorbar()
        cb.set_label("Covariance fraction")
        axis('square')

        subplot(gs[3,1])
        h=pcolor(calin.diagnostics.waveform.WaveformStatsVisitor.waveform_cov_frac(lg_wfs),
                cmap=get_cmap('bwr'));
        title("Waveform covariance matrix")
        xlabel('Sample number')
        ylabel('Sample number')
        h.set_clim((-1.0,1.0))
        cb = colorbar()
        cb.set_label("Covariance fraction [1]")
        axis('square')

        pdf.savefig()
        close(gcf())
    d = pdf.infodict()
    d['Title'] = 'Diagnostics'
    d['CreationDate'] = datetime.datetime.today()